In [19]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

ROOT = Path('../')
DATA_DIR = ROOT/'data'

CRS_NZGD49 = {'init': 'epsg:27200', 'no_defs': True}
CRS_NZTM = {'init': 'epsg:2193', 'no_defs': True}
CRS_WGS84 = {'init': 'epsg:4326'}

%matplotlib inline

In [42]:
# Car costs
# Use 30%/year diminishing value for cars taken from IRD depreciation calculator (https://interact1.ird.govt.nz/forms/depnrates/6f2b363212311e77353c7a1f373c6e2229472e70.continue;jsessionid=8C181312A1BA5BD98EE39396468C2E94)

def value(purchase_price, num_years, depreciation_rate=0.3):
    x = purchase_price
    n = num_years
    k = math.floor(n)
    kk = n - k
    r = depreciation_rate
    if n <= 0:
        d = 0
    else:
        d = sum([x*r**i for i in range(1, k + 1)]) + kk*x*r**(k + 1)
    return x - d

value(50679, 2)

30914.190000000002

# Explore Auckland Transport Journey Planner

In [71]:
def get_journey_auckland(orig, dest, departure_time=None, max_walk=2000):
    """
    INPUT:
        orig
            List; WGS84 longitude-latitude pair
        dest
            List; WGS84 longitude-latitude pair
        departure_time
            String; ISO 8601 datetime; e.g. '2017-06-01T07:30:00'

    """
    url = 'https://api.at.govt.nz/v2/public-restricted/journeyplanner/silverRailIVU/plan'
    fromLoc ='{!s},{!s}'.format(orig[1], orig[0])
    toLoc ='{!s},{!s}'.format(dest[1], dest[0])
    if departure_time is None:
        departure_time = dt.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    date = departure_time + '+12:00'  # Add UTC offset
    params = {
        'from': 'from',
        'to': 'to',
        'fromLoc': fromLoc,
        'toLoc': toLoc,
        'timeMode': 'A',
        'date': date, 
        'modes': 'BUS,TRAIN,FERRY',
        'operators': '',
        'optimize': 'QUICK',
        'maxWalk': max_walk,
        'maxChanges': '-1',
        'routes': '',
        'subscription-key': '323741614c1c4b9083299adefe100aa6',
    }
    r = requests.get(url, params=params)
    
    # Raise an error if bad request
    r.raise_for_status()

    return r.json()         

def get_fare_auckland(journey):
    try:
        fare = journey['response']['itineraries'][0]['fareAdult']/100
    except:
        fare = None
    return fare

In [72]:
# Test some
orig = [174.7433853149414, -36.85517522550505]
dest = [174.79625701904297, -36.82550066651677]
%time j = get_journey_auckland(orig, dest)
j

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 3.56 s


{'error': None,
 'response': {'engine': 'silverRailIVU',
  'itineraries': [{'duration': 4140000,
    'durationStr': 'an hour',
    'endTime': '2017-05-17T16:41',
    'endTimeStr': '4:41 pm',
    'fareAdult': 1000,
    'fareChild': 600,
    'fareError': None,
    'fareHopAdult': 645,
    'fareHopChild': 339,
    'fareHopTertiary': 498,
    'fareWarnings': {},
    'legs': [{'distance': '1162 metres',
      'distanceExact': 1162,
      'distanceStr': '1162 metres',
      'duration': 1020000,
      'durationStr': '17 min',
      'endLat': '-36.86023',
      'endLon': '174.75198',
      'endTime': '2017-05-17T15:49',
      'endTimeStr': '3:49 pm',
      'fareAdult': 0,
      'fareChild': 0,
      'fareHopAdult': 0,
      'fareHopChild': 0,
      'fareHopTertiary': 0,
      'from': 'from',
      'isFastLeg': False,
      'isFirst': True,
      'isLast': False,
      'legGeometry': {'length': 34,
       'points': 'zg}_Fcqpi`@G@e@}LGiAESlHkC^]NY`CsFfN_[pBlCKJ'},
      'messages': [],
      'mo

In [73]:
get_fare_auckland(j)

10.0

# Compute Auckland HOP fares

In [ ]:
path = DATA_DIR/'auckland'/'rental_area_points.geojson'
rap = gpd.read_file(str(path))
departure_time = '2017-06-01T07:30:00'

# Get all pairs of points excluding equal points
f = rap[['rental_area', 'geometry']].copy()
rows = [[o[0], o[1].coords[0], d[0], d[1].coords[0]] for o, d in product(f.values, f.values) if o[0] != d[0]]
f = pd.DataFrame(rows, columns=['orig_name', 'orig', 'dest_name', 'dest'])

print('This will take about {:02f} minutes'.format(f.shape[0]*3.6/60))

# Get journeys for each pair
rows = []
for __, row in f.iterrows():
    try:
        j = get_journey_auckland(row['orig'], row['dest'], departure_time=departure_time)
        fare = get_fare_auckland(j)
    except:
        fare = None
    rows.append([row['orig_name'], row['dest_name'], fare])

g = pd.DataFrame(rows, columns=['orig_name', 'dest_name', 'fare'])
path = DATA_DIR/'auckland'/'transit_costs.csv'
g.to_csv(str(path), index=False)
g


This will take about 570.360000 minutes
